In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [151]:
# Load dataset
df = pd.read_csv("../name_of_file.csv")

# Separate features and labels
X = df.drop(columns=['loan_status', 'confidence']).values
y = df['loan_status'].values

print(df['loan_status'].value_counts(normalize=True))

# Split by percentage
train_percentage = 0.8  # 80% train, 20% test
X_train, X_test, train_labels, test_labels = train_test_split(
    X, y, train_size=train_percentage, random_state=52, shuffle=True
)


# Step 3: create scaler
scaler = MinMaxScaler(feature_range=(0, np.pi))

# Step 4: fit scaler on your training set or just transform this vector
X_train = scaler.fit_transform(X_train.astype(float))  # or scaler.transform(x_array) if already fitted
X_test = scaler.fit_transform(X_test.astype(float))

X_train = [row.tolist() for row in X_train]
X_test = [row.tolist() for row in X_test]
train_labels = [row.tolist() for row in train_labels]
test_labels = [row.tolist() for row in test_labels]


loan_status
0    0.5
1    0.5
Name: proportion, dtype: float64


In [224]:
from qiskit.circuit.library import zz_feature_map

n_qubits = len(X_train[0])  # e.g., 10
feature_map = zz_feature_map(feature_dimension=n_qubits,entanglement='linear', reps=2)
#feature_map.draw("mpl")

In [225]:
x_sample = X_train[0]  # shape = (n_qubits,)
circuit = feature_map.assign_parameters(x_sample)
x_sample

[0.0,
 0.5235987755982988,
 1.1689647083124812,
 0.0,
 3.141592653589793,
 0.0,
 1.2014997443984536,
 0.0,
 0.0,
 1.1087974071493387]

In [238]:
circuit.draw("mpl")
fig = circuit.draw(output='mpl', style={"backgroundcolor": "None"}, fold=-1)  # empty labels for all qubits)  # returns a matplotlib Figure object

# Save to PNG
fig.savefig('circuit_linear_2.png', dpi=300, bbox_inches='tight', transparent=True)

In [215]:
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.primitives import StatevectorEstimator

In [216]:
lista_observables = []


for i in range(n_qubits):
    obs = SparsePauliOp.from_sparse_list(
        [("Z", [i], 1.0)],
        num_qubits=n_qubits
    )
    lista_observables.append(obs)
    print(f"Z on qubit {i}:", obs)


# we keep oonly correlated features: 

# loan_percent_income to loan_amnt

# loan_amnt ,loan_int_rate

# loan_int_rate and loan_perc_inc

# Two-qubit ZZ correlations
# corr = [(2,9), (9,6), (6,9)]
# 
# for (i,j) in corr:
#     pauli_str = ['I'] * n_qubits
#     pauli_str[i] = 'Z'
#     pauli_str[j] = 'Z'
#     obs = SparsePauliOp(''.join(pauli_str))
#     lista_observables.append(obs)

for i in range(n_qubits):
    for j in range(i + 1, n_qubits):
        pauli_str = ['I'] * n_qubits
        pauli_str[i] = 'Z'
        pauli_str[j] = 'Z'
        obs = SparsePauliOp(''.join(pauli_str))
        lista_observables.append(obs)

Z on qubit 0: SparsePauliOp(['IIIIIIIIIZ'],
              coeffs=[1.+0.j])
Z on qubit 1: SparsePauliOp(['IIIIIIIIZI'],
              coeffs=[1.+0.j])
Z on qubit 2: SparsePauliOp(['IIIIIIIZII'],
              coeffs=[1.+0.j])
Z on qubit 3: SparsePauliOp(['IIIIIIZIII'],
              coeffs=[1.+0.j])
Z on qubit 4: SparsePauliOp(['IIIIIZIIII'],
              coeffs=[1.+0.j])
Z on qubit 5: SparsePauliOp(['IIIIZIIIII'],
              coeffs=[1.+0.j])
Z on qubit 6: SparsePauliOp(['IIIZIIIIII'],
              coeffs=[1.+0.j])
Z on qubit 7: SparsePauliOp(['IIZIIIIIII'],
              coeffs=[1.+0.j])
Z on qubit 8: SparsePauliOp(['IZIIIIIIII'],
              coeffs=[1.+0.j])
Z on qubit 9: SparsePauliOp(['ZIIIIIIIII'],
              coeffs=[1.+0.j])


In [217]:
estimator = StatevectorEstimator()
 
pub = (
    circuit,  # ISA circuit
    lista_observables
)
 
job_result = estimator.run(pubs=[pub]).result()[0]

job_result.data.evs

array([-0.10399378,  0.01728082, -0.01306636, -0.10399378,  1.        ,
       -0.10399378, -0.00255096, -0.10399378, -0.10399378,  0.00103684,
       -0.01276843, -0.01276843,  0.14901401, -0.01276843,  0.00103684,
       -0.01276843,  0.27809947, -0.01927583, -0.01276843,  0.49975452,
        0.00174414,  0.49975452, -0.10399378,  0.49975452, -0.00151032,
       -0.02675195,  0.49975452,  0.00174414,  0.49975452, -0.10399378,
        0.49975452, -0.00151032, -0.02675195,  0.49975452,  0.00174414,
       -0.00255096,  0.00174414,  0.43916978,  0.20690837,  0.00174414,
       -0.10399378,  0.49975452, -0.00151032, -0.02675195,  0.49975452,
       -0.10399378, -0.01306636,  0.01728082, -0.10399378, -0.00151032,
       -0.02675195,  0.49975452,  0.09458763, -0.00151032, -0.02675195])

In [218]:
X_train_fan_out = []


for i in X_train:   
    circuit = feature_map.assign_parameters(i)
    pub = (
    circuit,  # ISA circuit
    lista_observables
    )
    job_result = estimator.run(pubs=[pub]).result()[0]

    nex_X_point = job_result.data.evs
    X_train_fan_out.append(nex_X_point)


In [204]:

X_train_fan_out = np.array(X_train_fan_out)

In [205]:
X_train_fan_out

array([[-1.62630326e-17, -1.50162001e-16, -7.84962373e-17, ...,
         5.74627151e-18,  2.16840434e-19,  1.31188463e-17],
       [-8.45677695e-18,  2.05998413e-18, -4.98732999e-17, ...,
        -2.71050543e-18,  4.55364912e-18, -3.19839641e-17],
       [ 7.80625564e-18, -2.27682456e-18, -5.81132364e-17, ...,
         1.65882932e-17, -2.16840434e-17,  4.66206934e-18],
       ...,
       [ 5.00901404e-17,  4.57533317e-17,  9.02056208e-17, ...,
        -4.11996826e-18, -2.10335221e-17, -1.58293517e-17],
       [ 3.68628739e-18, -1.89735380e-17,  1.99493200e-17, ...,
        -2.41777084e-17,  3.36102673e-17, -3.32850067e-17],
       [-2.29850861e-17,  2.98155597e-17, -6.41847686e-17, ...,
        -7.38341679e-17,  4.07660017e-17, -1.72388145e-17]],
      shape=(227, 55))

In [219]:
X_test_fan_out = []


for i in X_test:   
    circuit = feature_map.assign_parameters(i)
    pub = (
    circuit,  # ISA circuit
    lista_observables
    )
    job_result = estimator.run(pubs=[pub]).result()[0]

    nex_X_point = job_result.data.evs
    X_test_fan_out.append(nex_X_point)

In [220]:
X_test_fan_out = np.array(X_test_fan_out)

In [221]:
X_test_fan_out

array([[ 5.64591689e-04,  4.90369077e-07,  2.34858069e-02, ...,
         3.22153802e-02,  1.03104974e-01, -9.93920158e-02],
       [ 1.00000000e+00,  3.82737681e-02,  3.80044091e-03, ...,
         2.42853577e-02,  3.80044091e-03,  3.82737681e-02],
       [-3.61488716e-02, -3.72176073e-02, -2.36989311e-02, ...,
         3.02169238e-02,  2.82568018e-03,  1.16167926e-02],
       ...,
       [ 2.04490536e-01,  2.65246140e-01, -1.70335471e-01, ...,
         1.96325307e-02, -8.24223783e-02,  5.35297807e-02],
       [-1.37676236e-01, -2.59499442e-01, -4.64996487e-02, ...,
        -1.01525855e-01,  1.72512803e-02,  4.36973369e-02],
       [ 1.00000000e+00, -1.15098559e-01, -2.05675610e-07, ...,
         1.38272381e-05, -2.05675610e-07, -1.15098559e-01]],
      shape=(57, 55))

In [222]:
import xgboost
xgb_model = xgboost.XGBClassifier(objective = "binary:logistic", random_state=42)

xgb_model.fit(X_train_fan_out, train_labels)

y_pred = xgb_model.predict(X_test_fan_out)

In [223]:

from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision:", precision_score(test_labels, y_pred))
print("Recall:", recall_score(test_labels, y_pred))
print("F1:", f1_score(test_labels, y_pred))

Precision: 0.4838709677419355
Recall: 0.6
F1: 0.5357142857142857
